# Download data using Google Earth Engine

## Install libraries

In [113]:
# !pip install earthengine-api geemap python-dateutil

## Import libraries

In [114]:
import ee, geemap
from datetime import datetime
import math
from dateutil.relativedelta import relativedelta

## Authenticate with Earth Engine

In [115]:
ee.Authenticate(auth_mode="notebook")

True

In [116]:
ee.Initialize()

## Export maps to Drive

### Helper function to export images to Drive

In [117]:
# Define some export functions
def ee_export_img(img, folder, aoi, lat, lon, target_crs, crs_transform, scale=250, increasePrecision=True):
    if increasePrecision == True:
        img = img.toInt32()
    else:
        img = img.toInt16()
        


    task = ee.batch.Export.image.toDrive(
        image          = img,
        description    = f'{folder}_flood_{lat}_{lon}',
        fileNamePrefix = f'{folder}_flood_{lat}_{lon}',
        region         = aoi,
        scale          = scale,
        crs            = target_crs,
        crsTransform   = crs_transform,
        maxPixels      = 1e13,
        fileFormat     = 'GeoTIFF',
        formatOptions  = {'cloudOptimized': True},
        folder         = folder
    )
    
    task.start()

### CRS functions

In [118]:
# Utilities to calculate local CRS
def _wrap_lon_180(lon):
    """Wrap longitude to [-180, 180)."""
    lon = (lon + 180.0) % 360.0 - 180.0
    # Handle the edge case where lon = 180 exactly after modulo
    return -180.0 if lon == 180.0 else lon

def _centroid_of_bbox_minlon_maxlat_maxlon_minlat(bbox):
    """Centroid of [minLon, maxLat, maxLon, minLat], with dateline handling."""
    min_lon, max_lat, max_lon, min_lat = bbox
    # Detect dateline crossing (e.g. min_lon=170, max_lon=-170)
    crosses_date_line = min_lon > max_lon
    if crosses_date_line:
        width = (max_lon + 360.0) - min_lon
        lon_c = _wrap_lon_180(min_lon + width / 2.0)
    else:
        lon_c = (min_lon + max_lon) / 2.0
    lat_c = (min_lat + max_lat) / 2.0
    return lon_c, lat_c

def epsg_from_bbox(bbox):
    """
    Return an EPSG code string for a rectangle [minLon, maxLat, maxLon, minLat],
    choosing UTM based on the centroid (with Norway/Svalbard exceptions),
    and UPS for polar areas (>=84N or <=80S).
    """
    lon, lat = _centroid_of_bbox_minlon_maxlat_maxlon_minlat(bbox)
    lon = _wrap_lon_180(lon)

    # Polar: use UPS
    if lat >= 84.0:
        return "EPSG:32661"  # WGS 84 / UPS North
    if lat <= -80.0:
        return "EPSG:32761"  # WGS 84 / UPS South

    # Base UTM zone
    zone = int(math.floor((lon + 180.0) / 6.0)) + 1  # 1..60

    # Norway exception: 56N–64N and 3E–12E -> zone 32
    if 56.0 <= lat < 64.0 and 3.0 <= lon < 12.0:
        zone = 32

    # Svalbard exception: 72N–84N
    if 72.0 <= lat < 84.0:
        if   0.0 <= lon < 9.0:   zone = 31
        elif 9.0 <= lon < 21.0:  zone = 33
        elif 21.0 <= lon < 33.0: zone = 35
        elif 33.0 <= lon < 42.0: zone = 37

    epsg_base = 326 if lat >= 0 else 327
    return f"EPSG:{epsg_base}{zone:02d}"

### Date functions

In [119]:
# Function to subtract 6 months from a date
def subtract_six_months(s):
    d = datetime.strptime(s, "%Y-%m-%d").date()
    result = d - relativedelta(months=6)
    return result.isoformat()

def get_year(s):
    d = datetime.strptime(s, "%Y-%m-%d").date()
    return d.year

### Landsat filters

In [120]:
# Scale + mask helpers
def apply_scale(image):
    # Apply USGS scale/offset for optical SR bands
    optical = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    # Replace the original SR bands with scaled versions
    image = image.addBands(optical, None, True)
    return image

def mask_clear_l5(image):
    qa = image.select('QA_PIXEL')
    # Keep pixels that are NOT (dilated cloud OR cloud OR shadow OR snow) and NOT fill
    dilated_cloud = qa.bitwiseAnd(1 << 1).eq(0)
    cloud         = qa.bitwiseAnd(1 << 3).eq(0)
    shadow        = qa.bitwiseAnd(1 << 4).eq(0)
    snow          = qa.bitwiseAnd(1 << 5).eq(0)
    fill          = qa.bitwiseAnd(1 << 0).eq(0)

    clear = dilated_cloud.And(cloud).And(shadow).And(snow).And(fill)

    # Also drop radiometrically saturated pixels
    radsat_ok = image.select('QA_RADSAT').eq(0)

    return image.updateMask(clear.And(radsat_ok))

def mask_clear_l8(image):
    qa = image.select('QA_PIXEL')
    # Keep pixels that are NOT (dilated cloud OR cloud OR cirrus OR shadow OR snow) and NOT fill
    dilated_cloud = qa.bitwiseAnd(1 << 1).eq(0)
    cirrus        = qa.bitwiseAnd(1 << 2).eq(0)
    cloud         = qa.bitwiseAnd(1 << 3).eq(0)
    shadow        = qa.bitwiseAnd(1 << 4).eq(0)
    snow          = qa.bitwiseAnd(1 << 5).eq(0)
    fill          = qa.bitwiseAnd(1 << 0).eq(0)

    clear = dilated_cloud.And(cirrus).And(cloud).And(shadow).And(snow).And(fill)

    # Also drop radiometrically saturated pixels
    radsat_ok = image.select('QA_RADSAT').eq(0)

    return image.updateMask(clear.And(radsat_ok))


def add_ndvi_l5(image):
    # Landsat 5: NIR=SR_B4, RED=SR_B3
    ndvi = image.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')
    return image.addBands(ndvi)

def make_ndvi_l5(image): 
    return add_ndvi_l5(mask_clear_l5(apply_scale(image))).toFloat()

def add_ndvi_l8(image):
    # Landsat 8: NIR=SR_B5, RED=SR_B4
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return image.addBands(ndvi)

def make_ndvi_l8(image): 
    return add_ndvi_l8(mask_clear_l8(apply_scale(image))).toFloat()

### Flood locations to export

In [121]:
# Define an AOI using 2 coords
# We can store all these in an array since we need to download multiple flood events

bounding_boxes = [
    [90.5434433, 35.9011703, 113.1428100, 24.4745999], # 2005 Yangtze River Floods
    [73.3856213, 31.4590012, 94.8980266, 17.0118457], # 2007 Bangladesh Floods
    [-95.7559177, 21.6269405, -82.7325919, 10.7730460], # 2007 Tabasco Floods
    [-106.9219767, 53.5306078, -89.8607237, 45.4480160], # 2009 Red River Floods
    [-67.0098286, -14.1439741, -48.9536914, -34.6435289], # 2015 Paraguay and Uruguay El Nino Floods
    [-124.4166669, 43.1213794, -109.4103100, 22.8688613], # 2017 California Floods
    [99.1717616, 17.1286267, 101.4242872, 13.4298135], # 2011 Thailand Floods
    [4.7049263, 9.9129092, 8.1858980, 7.6222052], # 2018 Nigeria Floods
]

# Store the start date of the flood event to be used to look up datasets later
flood_dates = [
    ["2005-06-28", "2005-07-19"], # 2005 Yangtze River Floods
    ["2007-07-21", "2007-10-15"], # 2007 Bangladesh Floods
    ["2007-10-28", "2007-12-01"], # 2007 Tabasco Floods
    ["2009-03-17", "2009-03-19"], # 2009 Red River Floods
    ["2015-12-01", "2016-01-29"], # 2015 Paraguay and Uruguay El Nino Floods
    ["2017-02-16", "2017-03-03"], # 2017 California Floods
    ["2011-08-05", "2012-01-09"], # 2011 Thailand Floods
    ["2018-08-20", "2018-10-02"], # 2018 Nigeria Floods
]

In [122]:
# To prevent Google Drive from exceeding storage, manually export each location individually
i = 7

### Retrieve flood details

In [123]:
# Retrieve info about the flood event
min_lon, max_lat, max_lon, min_lat = bounding_boxes[i]
flood_start, flood_end = flood_dates[i]

# Additionally, it may be useful to lookup predictor variables using data prior to the flood, not during
flood_prior = subtract_six_months(flood_start)

# Build a proper rectangle: [minLon, minLat, maxLon, maxLat]
aoi = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat], geodesic=False)

target_scale = 25                       # metres
target_crs   = epsg_from_bbox(bounding_boxes[i])     # A CRS that has units in metres

In [124]:
# Clip the AOI to raster corners
proj = ee.Projection(target_crs).atScale(target_scale)

# AOI bounds in target projection
aoi_proj = aoi.transform(proj, 1)
bbox = aoi_proj.bounds(1, proj)

# bbox ring: [[minx, miny], [minx, maxy], [maxx, maxy], [maxx, miny], [minx, miny]]
ring = ee.List(bbox.coordinates().get(0))
minx = ee.Number(ee.List(ring.get(0)).get(0))
miny = ee.Number(ee.List(ring.get(0)).get(1))
maxx = ee.Number(ee.List(ring.get(2)).get(0))
maxy = ee.Number(ee.List(ring.get(2)).get(1))

# Snap to grid
x0 = minx.divide(target_scale).floor().multiply(target_scale)   # left  (UL x)
y0 = maxy.divide(target_scale).ceil().multiply(target_scale)    # top   (UL y)
x1 = maxx.divide(target_scale).ceil().multiply(target_scale)    # right
y1 = miny.divide(target_scale).floor().multiply(target_scale)   # bottom

# Snapped rectangle in target projection
snapped_region = ee.Geometry.Rectangle([x0, y1, x1, y0], proj, False)

# Affine transform (north-up)
transform = [target_scale, 0, x0, 0, -target_scale, y0]

### Load data sources

In [125]:
# DEM data (altitude)
dem = ee.Image('USGS/SRTMGL1_003').select('elevation')


# Derived slope data
slope = ee.Terrain.slope(dem).rename('slope')


# refer to https://code.earthengine.google.co.in/fa41d55186e6d708a4be367237353ff0
# We need to find the correct band that corresponds to the correct year
# To be safe, we subtract 1 from the year to get a LC map prior to the event
band = "b" + str((get_year(flood_start) - 1) - 2000 + 1)
worldcover = (ee.ImageCollection('projects/sat-io/open-datasets/GLC-FCS30D/annual')
                .filterBounds(aoi)
                .select(band)
                .mosaic())


# If we are looking at flood events more recent than 2012, we need Landsat 8 data for NDVI
if int(flood_start[:4]) > 2012:
    # Landsat 8 used to calculate NDVI
    ndvi = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
            .filterDate(flood_prior, flood_start)
            .filterBounds(aoi)
            .map(make_ndvi_l8)
            .select('NDVI')
            .median())
else:
    # Landsat 5 used to calculate NDVI
    ndvi = (ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
            .filterDate(flood_prior, flood_start)
            .filterBounds(aoi)
            .map(make_ndvi_l5)
            .select('NDVI')
            .median())


# Ground truth flood labels
flood_event = (ee.ImageCollection('GLOBAL_FLOOD_DB/MODIS_EVENTS/V1')
                .filterDate(flood_start, flood_end)
                .filterBounds(aoi)
                .mosaic())

flood_mask = (flood_event
                .select('flooded')
                .rename('label')
                .unmask(0))

# Rivers dataset for distance to rivers, choose only major rivers
rivers = ee.FeatureCollection('WWF/HydroSHEDS/v1/FreeFlowingRivers').filter('RIV_ORD <= 4').filterBounds(aoi)

distance = rivers.distance(
    searchRadius=200000,  # metres
    maxError=50
).rename('dist_m')

water = distance.unmask(200000)

### Reprojection, masking, clipping

In [126]:
def prep_img(img):
    return (img
            .reproject(crs=target_crs, scale=target_scale)
            .clip(snapped_region))

def prep_label(img):
    return (img
            .reproject(crs=target_crs, scale=250)
            .clip(snapped_region))

dem25 = prep_img(dem)
slope25 = prep_img(slope)
lc25 = prep_img(worldcover)
ndvi25 = prep_img(ndvi)
label250 = prep_label(flood_mask)
water25 = prep_img(water)

In [127]:
# To save on storage space, for maps with floats, we multiply them by 1000 and export as the map as ints
slope25 = slope25.multiply(1000)
ndvi25 = ndvi25.multiply(1000)
water25 = water25.multiply(1000)

### Exporting to Drive

In [128]:
ee_export_img(dem25, "dem", aoi, min_lon, max_lat, target_crs, crs_transform=transform, scale=target_scale, increasePrecision=False)
ee_export_img(slope25, "slope", aoi, min_lon, max_lat, target_crs, crs_transform=transform, scale=target_scale)
ee_export_img(lc25, "lc", aoi, min_lon, max_lat, target_crs, crs_transform=transform, scale=target_scale, increasePrecision=False)
ee_export_img(ndvi25, "ndvi", aoi, min_lon, max_lat, target_crs, crs_transform=transform, scale=target_scale)
ee_export_img(water25, "water", aoi, min_lon, max_lat, target_crs, crs_transform=transform, scale=target_scale)
ee_export_img(label250, "label", aoi, min_lon, max_lat, target_crs, crs_transform=transform, increasePrecision=False)

print('Export started.  Monitor your Tasks tab at https://code.earthengine.google.com/tasks')

Export started.  Monitor your Tasks tab at https://code.earthengine.google.com/tasks
